In [1]:
from code.data_module import CrowdTangleDataModule, inverse_transform
import pandas as pd
from tqdm import tqdm

In [2]:
config = {
    'batch_size': 32,
    'dataset_path': '../data/crowdtanglev2_filtered.csv',
    #'category_group_id': False # group id not encoded (we only have one page = rtnews)
}
data_module = CrowdTangleDataModule(config=config)
data_module.setup('')

In [3]:
df_dicts = []
df_dicts_unk = []
val_loader = data_module.val_dataloader()
#groupid2name = GroupId2Name(model.config)
for y_lst, encoded_texts_lst, category_vectors_lst, raw_batch in tqdm(iter(val_loader),total=len(val_loader)):
    for y, encoded_text, category_vector, raw in zip(y_lst, encoded_texts_lst['input_ids'], category_vectors_lst, raw_batch):
        decoded_categories = inverse_transform(category_vector.reshape(1, -1), config, data_module.class_encoder)
        row = {**{
            'gold label': float(y),
            'text_decoded': data_module.tokenizer.decode(encoded_text).replace(' [PAD]',''),
            'text': raw['Text']
        },**decoded_categories}
        if ('UNK' in row['text']):
            df_dicts_unk.append(row)
        else:
            df_dicts.append(row)

100%|██████████| 211/211 [00:08<00:00, 25.23it/s]


In [4]:
pd.DataFrame.from_dict(df_dicts)

,gold label,text_decoded,text,group_id,post_type,domain
0,0.528097,[CLS] https : / / inhomelandsecurity. com / us...,https://inhomelandsecurity.com/us-troops-may-h...,291686185301,Link,None
1,0.423097,[CLS] they say that in order for us to evolve ...,They say that in order for us to evolve and gr...,229114357132790,Native Video,facebook
2,0.035429,"[CLS] the people who've always called you a "" ...","THE PEOPLE WHO'VE ALWAYS CALLED YOU A ""CONSPI...",805671206178851,Photo,facebook
3,0.000000,[CLS] pediatric integrative medicine approachi...,pediatric integrative medicine approaching ch...,291686185301,Photo,facebook
4,0.000000,[CLS] time is of essence! [UNK] this is a re -...,Time is of essence!🌎 This is a re-share of a ...,229114357132790,Link,liberation-freedom
...,...,...,...,...,...,...
6718,0.150885,[CLS] when you are trying to protect your lung...,When you are trying to protect your lungs from...,291686185301,Native Video,facebook
6719,0.318863,"[CLS] ~ matshona dhliwayo "" you don't excel by...","~MATSHONA DHLIWAYO ""YOU DON'T EXCEL BY CONFOR...",805671206178851,Photo,facebook
6720,0.452654,[CLS] this has nothing to do with black people...,"This has nothing to do with black people, it's...",291686185301,Photo,facebook
6721,0.000000,[CLS] here ’ s what ’ s in those mystery seed ...,Here’s what’s in those mystery seed packages f...,291686185301,Link,None


In [6]:
[(r['text_decoded'],r['text']) for r in df_dicts]

[('[CLS] https : / / inhomelandsecurity. com / us - troops - may - have - role - in - mass - covid - 19 - vaccination - effort / us troops may have role in mass covid - 19 vaccination effort u. s. military medical personnel or national guard troops might assist in inoculating the public once a viable, safe vaccine is developed for covid - 19. [SEP]',
  'https://inhomelandsecurity.com/us-troops-may-have-role-in-mass-covid-19-vaccination-effort/  US Troops May Have Role in Mass COVID-19 Vaccination Effort U.S. military medical personnel or National Guard troops might assist in inoculating the public once a viable, safe vaccine is developed for COVID-19.'),
 ('[CLS] they say that in order for us to evolve and grow, we must question everything. so did you ever stop to question the disharmonious and irregular calendar that we are currently using? : d dreamspell galactic signature decoding the gregorian calendar, used and accepted without questioning by our society today, is a very irregular

In [11]:
len(df_dicts)

5